In [250]:
import numpy as np
import re
import math
import string
from copy import deepcopy
import itertools
from IPython.display import display, clear_output
import sys
from tqdm import tqdm
np.set_printoptions(suppress=True,linewidth=np.nan,threshold=sys.maxsize)

In [251]:
f = open("../input/day15.txt","r")
lines = f.read().splitlines()
len(lines)
TARGET_Y = 2000000

In [252]:
# TARGET_Y = 10
lines_test = [
"Sensor at x=2, y=18: closest beacon is at x=-2, y=15",
"Sensor at x=9, y=16: closest beacon is at x=10, y=16",
"Sensor at x=13, y=2: closest beacon is at x=15, y=3",
"Sensor at x=12, y=14: closest beacon is at x=10, y=16",
"Sensor at x=10, y=20: closest beacon is at x=10, y=16",
"Sensor at x=14, y=17: closest beacon is at x=10, y=16",
"Sensor at x=8, y=7: closest beacon is at x=2, y=10",
"Sensor at x=2, y=0: closest beacon is at x=2, y=10",
"Sensor at x=0, y=11: closest beacon is at x=2, y=10",
"Sensor at x=20, y=14: closest beacon is at x=25, y=17",
"Sensor at x=17, y=20: closest beacon is at x=21, y=22",
"Sensor at x=16, y=7: closest beacon is at x=15, y=3",
"Sensor at x=14, y=3: closest beacon is at x=15, y=3",
"Sensor at x=20, y=1: closest beacon is at x=15, y=3",
]

In [253]:
positions = []
for line in lines:
    if(not line): continue
    pos = list(map(int,re.match(r"Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)", line).groups()))
    positions.append([ (pos[0], pos[1]) ,  (pos[2], pos[3]) ])

In [254]:
max_pos_x = 0
max_pos_y = 0
min_pos_x = 0
min_pos_y = 0

for pos in positions:
    max_pos_x = max(pos[0][0], pos[1][0]) if max(pos[0][0], pos[1][0]) > max_pos_x else max_pos_x
    max_pos_y = max(pos[0][1], pos[1][1]) if max(pos[0][1], pos[1][1]) > max_pos_y else max_pos_y
    min_pos_x = min(pos[0][0], pos[1][0]) if min(pos[0][0], pos[1][0]) < min_pos_x else min_pos_x
    min_pos_y = min(pos[0][1], pos[1][1]) if min(pos[0][1], pos[1][1]) < min_pos_y else min_pos_y


In [255]:
print("x: ", min_pos_x, max_pos_x)
print("y: ", min_pos_y, max_pos_y)

x:  0 4064129
y:  -792661 3999960


In [256]:
from scipy.spatial.distance import cityblock

In [257]:
sensor_distances = []

min_radius_x = 0
max_radius_x = 0

for pos in positions:
    radius = cityblock(pos[0], pos[1])
    sensor_distances.append(radius)
    mi = pos[0][0] - radius
    ma = pos[0][0] + radius
    min_radius_x = mi if mi < min_radius_x else min_radius_x
    max_radius_x = ma if ma > max_radius_x else max_radius_x

min_radius_x, max_radius_x

(-1554506, 4961798)

In [258]:
x_covered_ranges = []


for i in range(len(positions)):
    sensor = positions[i][0]
    man_y = abs(sensor[1] - TARGET_Y)
    max_range_x = sensor_distances[i] - man_y
    if max_range_x < 0:
        continue
    x1 = sensor[0] - abs(max_range_x)
    x2 = sensor[0] + abs(max_range_x)
    
    x_covered_ranges.append([x1,x2])

In [259]:
def covered_to_final_range(covered_range):
    ranges_raw = sorted(covered_range)
    ranges_final =[ranges_raw[0]]

    for range_raw in ranges_raw[1:]:
        if range_raw[0] <= ranges_final[-1][1]:
            if range_raw[1] < ranges_final[-1][1]:
                continue
            ranges_final[-1][1] = range_raw[1]
        else:
            ranges_final.append(range_raw)

    return ranges_final

In [260]:
sum_ranges = 0
for range_f in covered_to_final_range(x_covered_ranges):
    sum_ranges += range_f[1] -  range_f[0]

sum_ranges

5073496

### Part 2

In [261]:
all_y_ranges = []

for y in tqdm(range(4000000)):

    covered_ranges = []
    for i in range(len(positions)):
        sensor = positions[i][0]
        man_y = abs(sensor[1] - y)
        max_range_x = sensor_distances[i] - man_y
        if max_range_x < 0:
            continue
        x1 = sensor[0] - max_range_x
        x2 = sensor[0] + max_range_x

        covered_ranges.append([x1,x2])

    all_y_ranges.append(covered_to_final_range(covered_ranges))    

100%|██████████| 4000000/4000000 [02:49<00:00, 23632.84it/s]


In [264]:
for y in range(len(all_y_ranges)):
    yr = all_y_ranges[y]
    if len(yr) > 1:
        print(yr)
        print(x)
        print(y)
        x = int(yr[0][1] + 1)
        sum = 4000000 * x  + y
        print(sum)

[[-589410, 3270297], [3270299, 4077403]]
3270298
2638237
13081194638237
